Отримання координат точок (де були зафіксовані пожежі або їх не було)

In [ ]:
#Ініціалізація (потрібна для подальшої роботи з інструментами GEE)
import ee
ee.Authenticate()
ee.Initialize(project='%Your project ID%')

In [21]:
#Віднімаємо рік від календарної дати (з урахуванням високосних років)
from datetime import datetime
from dateutil.relativedelta import relativedelta

def minus_year(current_date):
    older_date=datetime.strptime(current_date, '%Y-%m-%d')-relativedelta(years=1)
    return older_date.strftime('%Y-%m-%d')

In [22]:
#Номер дня в місяці за його номером в році
from datetime import timedelta

def day_of_month(year, day_of_year):
    base_date = datetime(year, 1, 1)  
    target_date = base_date + timedelta(days=day_of_year - 1)  
    day_of_month=target_date.day
    return day_of_month

In [23]:
#Отримання пожежних/не пожежних даних з квадрату ("вікна") з заданими координатами і іншими параметрами

import numpy as np
from skimage import morphology as skm
from skimage import measure

def GetDataFromSquare(start_date, end_date, coordX, coordY, deltaX, deltaY, scale, deltaX_small, deltaY_small):
    fire_dataset=ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate(start_date, end_date).select('BurnDate', 'LandCover')
    roi = ee.Geometry.BBox(coordX, coordY, coordX+deltaX, coordY+deltaY)
    pixelInfoBbox = fire_dataset.getRegion(roi, scale)
    pixelInfoBbox=pixelInfoBbox.getInfo()
    pixelInfoBbox=pixelInfoBbox[1:]
    matr_img=[]
    for i in range(len(pixelInfoBbox)):
        if i==0 or pixelInfoBbox[i][2]!=pixelInfoBbox[i-1][2]:
            matr_img.insert(0, [])
        if pixelInfoBbox[i][4] is None:
            matr_img[0].extend([367])
        else:
            matr_img[0].extend([pixelInfoBbox[i][4]])
    matr_trans = np.array(matr_img)#, dtype=np.uint8)
    matr_trans_morph=skm.black_tophat(matr_trans)#, square(3)))
    matr_trans_morph[matr_trans_morph!=0]=1
    labeled_matr, num_features = measure.label(matr_trans_morph, connectivity=1, return_num=True)
    if num_features>0:
        height=len(labeled_matr)
        width=len(labeled_matr[0])
        for i in range(1, num_features+1):
            rows, cols = np.where(labeled_matr == i)
            min=367
            for row, col in zip(rows, cols):
                num=(height-row-1)*width+col
                if pixelInfoBbox[num][4]<min:
                    min=pixelInfoBbox[num][4]
            for row, col in zip(rows, cols):
                if row!=0 and col!=0 and row!=height-1 and col!=width-1:
                    num=(height-row-1)*width+col
                    if day_of_month(Year, pixelInfoBbox[num][4])>3:
                        if pixelInfoBbox[num][5]>=50 and pixelInfoBbox[num][5]<=90:
                            if pixelInfoBbox[num][4]==min:
                                points.append([pixelInfoBbox[num][0].replace("_", "-"), pixelInfoBbox[num][1], pixelInfoBbox[num][2], pixelInfoBbox[num][4]])
                                roi_old = ee.Geometry.BBox(pixelInfoBbox[num][1]-deltaX_small, pixelInfoBbox[num][2]-deltaY_small, pixelInfoBbox[num][1]+deltaX_small, pixelInfoBbox[num][2]+deltaY_small)
                                matched=False
                                older_start_date=start_date
                                older_end_date=end_date
                                while not matched:
                                    older_start_date=minus_year(older_start_date)
                                    older_end_date=minus_year(older_end_date)
                                    fire_dataset_old=ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate(older_start_date, older_end_date).select('BurnDate')
                                    pixelInfoBbox_old = fire_dataset_old.getRegion(roi_old, scale)
                                    pixelInfoBbox_old=pixelInfoBbox_old.getInfo()
                                    pixelInfoBbox_old=pixelInfoBbox_old[1:]
                                    for pib_old in pixelInfoBbox_old:
                                        if pib_old[1]==pixelInfoBbox[num][1] and pib_old[2]==pixelInfoBbox[num][2] and pib_old[4] is None:
                                            points.append([pib_old[0].replace("_", "-"), pib_old[1], pib_old[2], pib_old[4]])
                                            matched=True
                                            break
                                        elif pib_old[1]==pixelInfoBbox[num][1] and pib_old[2]==pixelInfoBbox[num][2] and pib_old[4] is not None:
                                            break

In [ ]:
#Збір даних по лісах України
points=[]
scale_meters=1000
Xdelta=0.5
Ydelta=0.25
Xdelta_small=0.01
Ydelta_small=0.01
border = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Ukraine'))
ukr = ee.Feature(border.first())
ukr_geometry=ukr.geometry()

for Year in range(2017, 2021):
    for Month in range(3, 12):
        if Month<9:
            date_start=str(Year)+'-0'+str(Month)+'-01'
            date_end=str(Year)+'-0'+str(Month+1)+'-01'
        elif Month==9:
            date_start=str(Year)+'-0'+str(Month)+'-01'
            date_end=str(Year)+'-'+str(Month+1)+'-01'
        else:
            date_start=str(Year)+'-'+str(Month)+'-01'
            date_end=str(Year)+'-'+str(Month+1)+'-01'
        print(date_start, date_end)
        Ycoord=49.9
        while Ycoord<52.4:
            Xcoord=23
            while Xcoord<35:
                bbox=ee.Geometry.BBox(Xcoord, Ycoord, Xcoord+Xdelta, Ycoord+Ydelta)
                if bbox.intersects(ukr_geometry, maxError=1).getInfo():
                    GetDataFromSquare(date_start, date_end, Xcoord, Ycoord, Xdelta, Ydelta, scale_meters, Xdelta_small, Ydelta_small)
                Xcoord+=0.5
            Ycoord+=0.25
        Ycoord=47.65
        while Ycoord<49.9:
            Xcoord=22
            while Xcoord<26.5:
                bbox=ee.Geometry.BBox(Xcoord, Ycoord, Xcoord+Xdelta, Ycoord+Ydelta)
                if bbox.intersects(ukr_geometry, maxError=1).getInfo():
                    GetDataFromSquare(date_start, date_end, Xcoord, Ycoord, Xdelta, Ydelta, scale_meters, Xdelta_small, Ydelta_small)
                Xcoord+=0.5
            Ycoord+=0.25

In [ ]:
#Збір даних по лісах Польші
points=[]
scale_meters=1000
Xdelta=0.5
Ydelta=0.25
Xdelta_small=0.01
Ydelta_small=0.01
border = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Poland'))
pol = ee.Feature(border.first())
pol_geometry=pol.geometry()

for Year in range(2017, 2021):
    for Month in range(3, 12):
        if Month<9:
            date_start=str(Year)+'-0'+str(Month)+'-01'
            date_end=str(Year)+'-0'+str(Month+1)+'-01'
        elif Month==9:
            date_start=str(Year)+'-0'+str(Month)+'-01'
            date_end=str(Year)+'-'+str(Month+1)+'-01'
        else:
            date_start=str(Year)+'-'+str(Month)+'-01'
            date_end=str(Year)+'-'+str(Month+1)+'-01'
        print(date_start, date_end)
        Ycoord=49
        while Ycoord<55:
            Xcoord=14.1
            while Xcoord<24.1:
                bbox=ee.Geometry.BBox(Xcoord, Ycoord, Xcoord+Xdelta, Ycoord+Ydelta)
                if bbox.intersects(pol_geometry, maxError=1).getInfo():
                    GetDataFromSquare(date_start, date_end, Xcoord, Ycoord, Xdelta, Ydelta, scale_meters, Xdelta_small, Ydelta_small)
                Xcoord+=0.5
            Ycoord+=0.25

In [ ]:
#Збереження даних у CSV-файл для Польщі (для України - аналогічно)
npoints=np.array(points, dtype=object)
file_path = "points_poland.csv"
np.savetxt(file_path, npoints, delimiter=',', fmt='%s')

In [ ]:
#Завантаження отриманих даних до сховища на сервері Google Cloud для Польщі (для України - аналогічно)
from google.cloud import storage
import json

client = storage.Client()

bucket = client.get_bucket('%your_bucket_name%')

blob = bucket.blob('points_poland.json')

data_json = json.dumps(points)

blob.upload_from_string(data_json)